In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Laser Link Analysis\n",
        "This notebook plots normalized flux vs propagation distance (or time)\n",
        "for each wavelength and extracts attenuation coefficients."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "import numpy as np\n",
        "import pandas as pd\n",
        "import matplotlib.pyplot as plt\n",
        "import glob\n",
        "import re\n",
        "\n",
        "csv_files = glob.glob('../data/flux_vs_time_lambda_*_dist_*.csv')\n",
        "\n",
        "records = []\n",
        "for cf in csv_files:\n",
        "    # Example filename format: flux_vs_time_lambda_532_dist_2.00.csv\n",
        "    match = re.search(r'lambda_(\\d+)_dist_(\\d+\\.\\d+)', cf)\n",
        "    if not match:\n",
        "        continue\n",
        "    lam = float(match.group(1))\n",
        "    dist = float(match.group(2))\n",
        "\n",
        "    # Read the single final flux row in this naive example\n",
        "    df = pd.read_csv(cf)\n",
        "    # We'll assume last row is the final flux\n",
        "    final_flux = df.iloc[-1]['flux']\n",
        "    records.append((lam, dist, final_flux))\n",
        "\n",
        "df_all = pd.DataFrame(records, columns=['lambda_nm','distance_m','flux'])\n",
        "display(df_all.head())\n",
        "\n",
        "# Example: group by lambda and plot flux vs distance\n",
        "for lam, grp in df_all.groupby('lambda_nm'):\n",
        "    plt.plot(grp['distance_m'], grp['flux']/grp['flux'].max(), 'o-', label=f'{lam} nm')\n",
        "\n",
        "plt.xlabel('Distance (m)')\n",
        "plt.ylabel('Normalized Flux')\n",
        "plt.title('Flux vs Distance for Different Wavelengths')\n",
        "plt.legend()\n",
        "plt.show()\n",
        "\n",
        "# Simple exponential fit: flux ~ exp(-alpha * distance)\n",
        "fit_results = []\n",
        "for lam, grp in df_all.groupby('lambda_nm'):\n",
        "    x = grp['distance_m'].values\n",
        "    y = grp['flux'].values\n",
        "    # Normalize\n",
        "    y0 = y/y.max()\n",
        "    # ln(y0) = -alpha * x\n",
        "    # alpha = - slope\n",
        "    # do a linear fit\n",
        "    logy = np.log(y0 + 1e-15)\n",
        "    A = np.vstack([x, np.ones(len(x))]).T\n",
        "    m, c = np.linalg.lstsq(A, logy, rcond=None)[0]\n",
        "    alpha_est = -m\n",
        "    fit_results.append((lam, alpha_est))\n",
        "\n",
        "df_alpha = pd.DataFrame(fit_results, columns=['lambda_nm','alpha'])\n",
        "display(df_alpha)\n"
      ],
      "execution_count": null,
      "outputs": []
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
